In [ ]:
#Prediction of Sales for the next month on the basis of given dataset 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#1.Data Analysis

In [ ]:
item_categories= pd.read_csv("../input/competitive-data-science-predict-future-sales/item_categories.csv")

In [ ]:
item_categories.head()


In [ ]:
items= pd.read_csv("../input/competitive-data-science-predict-future-sales/items.csv")

In [ ]:
items.head()

In [ ]:
sample_submission = pd.read_csv("../input/competitive-data-science-predict-future-sales/sample_submission.csv")

In [ ]:
sample_submission.head()

In [ ]:
#Traing data 
train= pd.read_csv("../input/competitive-data-science-predict-future-sales/sales_train.csv")
train.head()

In [ ]:
#2.EDA

In [ ]:
train.plot()

In [ ]:
train["item_price"].plot()

In [ ]:
test =pd.read_csv("../input/competitive-data-science-predict-future-sales/test.csv")
test.head()

In [ ]:
shops = pd.read_csv("../input/competitive-data-science-predict-future-sales/shops.csv")
shops.head()

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(x="item_price",y="item_cnt_day", data= train, marker='o')
plt.show()

In [ ]:
import seaborn as sns


ax = sns.violinplot(x=train["item_cnt_day"])

In [ ]:
ax = sns.violinplot(x=train["item_price"])

In [ ]:
z = plt.hist(x="item_price", data=train)
plt.show()

In [ ]:
train["item_price"].max()

In [ ]:
test_shops = test.shop_id.unique()
train = train[train.shop_id.isin(test_shops)]
test_items = test.item_id.unique()
train = train[train.item_id.isin(test_items)]

In [ ]:
test_shops

In [ ]:
train["date_block_num"].max()

In [ ]:
z = plt.hist(x="date_block_num", data=train)
plt.show()

In [ ]:
#3.Tranforamton of data

In [ ]:
train['date'] = pd.to_datetime(train['date'],format = '%d.%m.%Y')

In [ ]:
train.head(3)

In [ ]:
# Plot of Pivot data using required features from dataset
data= train.pivot_table(index=['shop_id','item_id'], values=['item_cnt_day'],columns=['date_block_num'], fill_value=0, aggfunc='sum')
data.reset_index(inplace=True)
#Set indices so that data can be easily manipulated

In [ ]:
data.head(10)

In [ ]:
#Left join is performed to keep the data which is needed to predict
data =pd.merge(test,data,on=["item_id", 'shop_id'],how='left')

In [ ]:
data.head(10)

In [ ]:
data.fillna(0,inplace=True)
data.head()

In [ ]:
data.drop(['shop_id', 'item_id',"ID"], inplace=True, axis=1)

In [ ]:
data.head(10)

In [ ]:
#4.Splitting train and test data

In [ ]:
# Train and test dataset
X_train =np.expand_dims(data.values[:,:-1], axis=2)
y_train= data.values[:,-1]
X_test =np.expand_dims(data.values[:,1:], axis=2)
print(X_train.shape,y_train.shape, X_test.shape)

In [ ]:
#5.Chossing method for prediction

In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from keras.datasets import mnist
# from extra_keras_datasets import emnist
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense


In [ ]:
model_lstm = Sequential()
model_lstm.add(LSTM(units =64,input_shape =(X_train.shape[1], X_train.shape[2])))
model_lstm.add(Dropout(0.4))
model_lstm.add(Dense(1))
model_lstm.compile(loss= "mse", optimizer='adam', metrics= ['mean_squared_error'])
model_lstm.summary()

In [ ]:
history_lstm = model_lstm.fit(X_train,y_train,batch_size=4096,epochs=10)

In [ ]:
#plot loss and accuracy 
plt.plot(history_lstm.history['loss'],color='b',label="training loss")
plt.legend(loc='best',shadow=True)

In [ ]:
submission_file = model_lstm.predict(X_test)
submission_file = submission_file.clip(0,20)
submission = pd.DataFrame({'ID' : test['ID'], 'item_cnt_month':submission_file.ravel()})
submission.to_csv('sub_file.csv',index=False, header=1)

In [ ]:
submission.head()

In [ ]:
submission['item_cnt_month'].max()

In [ ]:
submission.groupby(['ID'])['item_cnt_month'].transform(max) 